In [3]:
import numpy as np
import math
import random
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

In [4]:
from Arm_Lib import Arm_Device
Arm = Arm_Device()  # Get DOFBOT object

FileNotFoundError: [Errno 2] No such file or directory

In [5]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

Defines Arm related functions for moving the arm. 

In [6]:
def set_angles(arm, angles, t = angles*np.pi/180*6):
    """Sets all the angles of arm
    Args:
        arm - the real robot
        angles - the angles to set
        t - the amount of time to move each angle
    """
    for joint, (angle, movetime) in enumerate(zip(angles, t)):
        arm.Arm_serial_servo_write(joint+1, angle, movetime)

def readAllActualJointAngles(arm):
    q = np.array([arm.Arm_serial_servo_read(id) for id in range(1,6)])
    return q

In [7]:
# search for gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device} (device)')

Using cuda (device)


Define the architechure for the Actor and Critic Networks

Initialize Dofbot

In [10]:
ex = np.array([1,0,0])
ey = np.array([0,1,0])
ez = np.array([0,0,1])
l0 = 0.061 # base to servo 1
l1 = 0.0435 # servo 1 to servo 2
l2 = 0.08285 # servo 2 to servo 3
l3 = 0.08285 # servo 3 to servo 4
l4 = 0.07385 # servo 4 to servo 5
l5 = 0.05457 # servo 5 to gripper
P01 = ( l0 + l1 ) * ez 
P12 = np.zeros (3) # translation between 1 and 2 frame in 1 frame
P23 = l2 * ex # translation between 2 and 3 frame in 2 frame
P34 = - l3 * ez # translation between 3 and 4 frame in 3 frame
P45 = np.zeros (3) # translation between 4 and 5 frame in 4 frame
P5T = -( l4 + l5 ) * ex 
print(P01,P12,P23,P34,P45,P5T)

P = np.array([P01, P12, P23, P34, P45, P5T]).T
H = np.array([ez, -ey, -ey, -ey, -ex]).T
print(P,H)
limits = np.array([0,180] * 6).reshape(6, 2)
limits[4, :] = [0, 270]
print(limits)

[0.     0.     0.1045] [0. 0. 0.] [0.08285 0.      0.     ] [-0.      -0.      -0.08285] [0. 0. 0.] [-0.12842 -0.      -0.     ]
[[ 0.       0.       0.08285 -0.       0.      -0.12842]
 [ 0.       0.       0.      -0.       0.      -0.     ]
 [ 0.1045   0.       0.      -0.08285  0.      -0.     ]] [[ 0  0  0  0 -1]
 [ 0 -1 -1 -1  0]
 [ 1  0  0  0  0]]
[[  0 180]
 [  0 180]
 [  0 180]
 [  0 180]
 [  0 270]
 [  0 180]]


In [11]:
from Robot import UR5Arm
dofbot = UR5Arm(P, H, limits)

During the Training Cycle, the following steps are performed for each epoch:
1. Initialize the Robot to a random state.
2. select a random end effector position and orientation that
   converges to valid joint angles using inverse kinematics
3. Run the following in each training epoch that lasts 10 seconds per iteration
4. The robot can takes 1 action in a discrete action space for each joint:
   - the DQN network outputs a value between -10 and 10 and the arm is commanded to
     move each servo that angle amount with velocity V
5. Repeat

First we need to establish the bounds for this robot. 
We move the robot to the home position and measure the maximal length the end effector can reach.

In [1]:
from path_planning import measure_actual_position
from kinematics import fwdkin, invkin

dofbot = Arm_Device()
home_angles = np.ones(6)*90
time_per_angle = 6  # in milliseconds
print(home_angles)
set_angles(dofbot, home_angles, home_angles * time_per_angle)

radius = dofbot.Arm_serial
def valid_configurations(arm: Arm):
    pass

NameError: name 'Arm_Device' is not defined

In [38]:
def generate_P_from_limits(limits):
    """Tests the target EE configuration process
    Generates a random end effector point based on Position vector limits
    Args:
        limits is a tuple of the smallest and largest norms of P_ees generated
    """
    target_norm = np.random.uniform(limits[0], limits[1])
    print(target_norm)
    random = np.random.rand(3)
    norm_vec = np.linalg.norm(random)
    out = random * target_norm/norm_vec
    return out

# Set the robot to the home position

In [ ]:
# define the neural networks
"""
The neural network learns an approximation of the value function
with a neural network
"""
class DQN(nn.Module):
    def __init__(self):
        super().__init()
        
    def forward(self, x):
        pass


In [ ]:
limits = test

In [ ]:
import random
from collections import namedtuple, deque

class ReplayBuffer:
    def __init__(self, action_size, buffer_size, batch_size, seed=0):
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)

        states = np.vstack([e.state for e in experiences if e is not None])
        actions = np.vstack([e.action for e in experiences if e is not None])
        rewards = np.vstack([e.reward for e in experiences if e is not None])
        next_states = np.vstack([e.next_state for e in experiences if e is not None])
        dones = np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        return len(self.memory)

In [ ]:

def train(episodes: int, virtual_arm: UR5Arm, real_arm: Arm):

    ms_per_angle = 6
    # coefficient for time weight
    t_coeff = 0.1
    q_network = DQN().to(device)

    # optimization and ML constants
    for ep in episodes:
        # select target destination for arm
        """
        target_end_effector is used to evaluate the model.
        The farther the euclidean distance from the end effector,
        the lower the reward.
        """
        collision = True
        target_angles = np.zeros(5,1)
        while not collision:
            target_angles = np.random.rand(5, 1) * virtual_arm.get_limits()[:,1].flatten())
            collision = collision_check(arm, target_angles)
        target_end_effector_pose = fwdkin(dofbot, target_angles)

        """
        the state are the current servo
        """
        start_collision = True
        starting_angles = np.zeros(5,1)
        while not start_collision:
            starting_angles = np.random.rand(5, 1) * virtual_arm.get_limits()[:,1].flatten())
            collision = collision_check(arm, starting_angles)
        
        # initialize the arm to a random state. 
        set_angles(real_arm, starting_angles, [1000] * 5)

        # allow the arm to move for 5 seconds
        time_start = time.time()
        elapsed_time = 0
        while(elapsed_time - time_start < 10):
            curr_angles = readAllActualJointAngles()
            current_end_effector_pose = fwdkin(dofbot, curr_angles)
    
            observations = curr_angles, current_end_effector_pose, target_end_effector_pose
            dq = q_network(observations)

            next_angles = curr_angles + dq
            """
            TO DO 
            check the joint limits and 
            penalize if the network tried to go over the joint limits
            """
            set_angles(real_arm, next_angles, dq * ms_per_angle)
            
            loss = np.linalg.norm(target_angles - curr_angles) + tcoef * elapsed_time
            """
            TO DO
            gradient descent on the q network here based on loss
            """          
        
